In [2]:
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler

from modis_tools.auth import add_earthdata_netrc, remove_earthdata_netrc

from datetime import datetime, timedelta
from pathlib import Path
import yaml

### Use modis-tools to search and download MODIS granules

In [2]:
# Create an entry for Earthdata in the ~/.netrc file, only needs to be run once

#username = ""
#password = ""
#add_earthdata_netrc(username, password)

In [3]:
# Create a session
session = ModisSession()

In [4]:
# Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)

# Choose a collection to query
#collections = collection_client.query(short_name="MYD02HKM", version="6.1") # Aqua 500m calibrated radiances
#collections = collection_client.query(short_name="MYD03", version="6.1") # Aqua geolocation fields
#collections = collection_client.query(short_name="MYD29", version="61") # Aqua sea ice
#collections = collection_client.query(short_name="MYD06_L2", version="6.1") # Aqua clouds

collections = collection_client.query(short_name=["MYD02HKM", "MYD03"], version="6.1")
#collections = collection_client.query(short_name=["MYD021KM", "MYD03"], version="6.1")

#collections = collection_client.query(short_name="MOD44W", version='061') # MODIS land/water mask


collections

[Collection(id='C1379758778-LAADS', title='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', dataset_id='MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 500m', coordinate_system='CARTESIAN', time_start='2002-07-04T00:45:00.000Z', updated=None, links=[CollectionLink(hreflang='en-US', href=AnyUrl('https://doi.org/10.5067/MODIS/MYD02HKM.061', scheme='https', host='doi.org', tld='org', host_type='domain', path='/10.5067/MODIS/MYD02HKM.061'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/default/files/file_attachments/M1054D_PUG_083112_final.pdf'), type=None), CollectionLink(hreflang='en-US', href=AnyUrl('https://mcst.gsfc.nasa.gov/sites/mcst.gsfc/files/file_attachments/MODIS_L1B_ATBD_ver4.pdf', scheme='https', host='mcst.gsfc.nasa.gov', tld='gov', host_type='domain', path='/sites/mcst.gsfc/files

In [47]:
# Criteria to filter the selected granules via spatial and temporal parameters
# e.g.
# nigeria_bbox = [2.1448863675, 4.002583177, 15.289420717, 14.275061098]
# nigeria_granules = granule_client.query(start_date="2016-01-01", end_date="2018-12-31", bounding_box=nigeria_bbox)

# See https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html for more options

# Date range:
start_date = "2022-01-01"
end_date = "2022-12-31"

# Spatial extent:
labsea = [-65.0, 45.0, -40.0, 70.0] # bounding_box for Labrador Sea
points = ["-55.0,53.75", "-64.25,61.5", "-53.25,64.25", "-44.0,59.25"] # four approximate 'corners' of the Labrador Sea
points2 = ["-56.0,52.5", "-68.0,60.0", "-53.0,66.0", "-41.0,60.0"] # four corners of a larger area
points3 = ["-64.0,55.0", "-44.0,63.0"] # two ends of a line bisecting the Labrador Sea

# NB point co-ordinate is in lon, lat (opposite to earthdata web search!)
# Default option for point search is AND, i.e. all points will be within each granule

aoi_corners = [[-65,45], [10,45],[10,65],[65,65],[65,82],[-30,82], [-30,70],[-65,70],[-65,45]]
bb1_corners = [[-65,45],[-10,45],[-10,70],[-65,70],[-65,45]]
bb2_corners = [[-30,60],[10,60],[10,82],[-30,82],[-30,60]]
bb3_corners = [[10,65],[65,65],[65,82],[10,82],[10,65]]

bb1 = ",".join([str(x) for x in [bb1_corners[0][0], bb1_corners[0][1], bb1_corners[2][0], bb1_corners[2][1]]])
bb2 = ",".join([str(x) for x in [bb2_corners[0][0], bb2_corners[0][1], bb2_corners[2][0], bb2_corners[2][1]]])
bb3 = ",".join([str(x) for x in [bb3_corners[0][0], bb3_corners[0][1], bb3_corners[2][0], bb3_corners[2][1]]])

In [26]:
bb1

'-65,45,10,70'

In [21]:
# Dictionary of arguments to be passed to each query
query_args = dict(start_date=start_date, end_date=end_date, 
                  day_night_flag="day", point=points2)

In [48]:
query_args = {'start_date':start_date, 'end_date':end_date, 
                   'day_night_flag':"day", 'bounding_box':[bb1,bb2,bb3], 'options[bounding_box][or]':"true"}
                  

In [24]:
# Query args for the MOD44W collection

# kwargs must be passed as dict if any options are to be used, e.g OR instead of AND
# e.g. 
# labsea2022_granules = granule_client.query(start_date=start_date, end_date=end_date, 
#                                           **{"point":["-52.5,57.5", "-62.4,57.5"], "options[point][or]":"true",
#                                              "day_night_flag":"day"})

query_args = {'start_date':start_date, 'end_date':end_date, 
                  'day_night_flag':"day", 'point':points2, 'options[point][or]':"true"}



In [49]:
# Query the selected collection for granules (only one collection at a time)
granule_client_data = GranuleApi.from_collection(collections[0], session=session)

# Get list of granules fitting given criteria:
data_granules = granule_client_data.query(**query_args) 

# Check how many granules before downloading!
data_granules = list(data_granules) # make it a list as generator can only be used once
len(data_granules)

6076

In [7]:
# Get geolocation granules for the same time period
granule_client_geo = GranuleApi.from_collection(collections[1], session=session)
geo_granules = granule_client_geo.query(**query_args)
geo_granules = list(geo_granules)
len(geo_granules)

71

In [8]:
# Check name of scene/granule is parsed correctly! e.g. 'MYD02HKM.A2022352.1640'
dict(data_granules[0])['producer_granule_id'][:22]

'MYD021KM.A2022352.1640'

In [7]:
data_granules

[Granule(id='G2571028161-NSIDC_ECS', title='SC:MYD29.061:257519593', dataset_id='MODIS/Aqua Sea Ice Extent 5-Min L2 Swath 1km V061', coordinate_system='GEODETIC', time_start='2022-12-18T16:40:00.000Z', updated=datetime.datetime(2022, 12, 24, 9, 3, 14, 414000, tzinfo=datetime.timezone.utc), links=[GranuleLink(hreflang='en-US', href=AnyUrl('https://n5eil01u.ecs.nsidc.org/DP4/MOSA/MYD29.061/2022.12.18/MYD29.A2022352.1640.061.2022358154045.hdf', scheme='https', host='n5eil01u.ecs.nsidc.org', tld='org', host_type='domain', path='/DP4/MOSA/MYD29.061/2022.12.18/MYD29.A2022352.1640.061.2022358154045.hdf'), type='application/x-hdfeos', inherited=None), GranuleLink(hreflang='en-US', href=AnyUrl('https://n5eil01u.ecs.nsidc.org/DP0/BRWS/Browse.001/2022.12.24/BROWSE.MYD29.A2022352.1640.061.2022358154045.1.jpg', scheme='https', host='n5eil01u.ecs.nsidc.org', tld='org', host_type='domain', path='/DP0/BRWS/Browse.001/2022.12.24/BROWSE.MYD29.A2022352.1640.061.2022358154045.1.jpg'), type='image/jpeg', i

In [9]:
# Make dictionary of scenes with data and geolocation filenamess

scenes = {}

for granule, geogranule in zip(data_granules, geo_granules):
    filename = dict(granule)['producer_granule_id']
    scenes[filename[:22]] = {'data':filename, 'geolocation':dict(geogranule)['producer_granule_id']}
    # maybe should put a check here to ensure the geolocation file is 
    # for the same date as the data file in case any missing data

In [8]:
# Specify download folder

folder = "Aqua1kmLabSea2022"
filepath = Path("/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/"+folder)
filepath.mkdir(exist_ok=True, parents=True)

In [18]:
# File path for the MOD44W water mask data
#filepath = "/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/MODIS_WaterMask"

In [17]:
# Write scene_ids and filenames to yaml file

meta_filepath = filepath / "meta"
meta_filepath.mkdir(exist_ok=True, parents=True)

with open(meta_filepath / 'scene_ids.yaml', 'w') as f:
    yaml.dump(scenes, f)


In [8]:
# Download the pics

data_filepath = filepath / "pics"
data_filepath.mkdir(exist_ok=True, parents=True)

GranuleHandler.download_from_granules(data_granules, session, path=str(data_filepath), threads=10, ext='jpg')

Downloading:   0%|          | 0/71 [00:00<?, ?file/s]

[PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022352.1640.061.2022355090005.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022343.1650.061.2022344180910.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022331.1630.061.2022332170548.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022322.1640.061.2022324011554.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022319.1610.061.2022321102137.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022316.1540.061.2022321092936.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022310.1620.061.2022312103754.jpg'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/pics/MYBRGB.A2022307.1550.061.2022309012310.jpg'),
 PosixPath('/hom

In [10]:
# Download some test granules

test_filepath = filepath / "test_downloads"
test_filepath.mkdir(exist_ok=True, parents=True)

GranuleHandler.download_from_granules(data_granules[0:2], session, path=str(test_filepath), threads=10)

Downloading:   0%|          | 0/2 [00:00<?, ?file/s]

[PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/test_data/MYD29.A2022316.1540.061.2022317195707.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/test_data/MYD29.A2022310.1620.061.2022311202316.hdf')]

In [ ]:
# Download the data granules

data_filepath = filepath / "l1b_data"
data_filepath.mkdir(exist_ok=True, parents=True)

GranuleHandler.download_from_granules(data_granules, session, path=str(data_filepath), threads=10)

In [12]:
# Download the geolocation granules

geo_filepath = filepath / "geoloc"
geo_filepath.mkdir(exist_ok=True, parents=True)

GranuleHandler.download_from_granules(geo_granules, session, path=str(geo_filepath), threads=10)

Downloading:   0%|          | 0/71 [00:00<?, ?file/s]

[PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022352.1640.061.2022353162327.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022343.1650.061.2022344151151.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022331.1630.061.2022332152535.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022322.1640.061.2022323233701.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022319.1610.061.2022320182159.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022316.1540.061.2022317155640.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022310.1620.061.2022311152222.hdf'),
 PosixPath('/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/Aqua1kmLabSea2022/geoloc/MYD03.A2022307.1550.061.2022308152844.hdf'),
 PosixPa